In [1]:
import numpy as np
from page import Page
from allocators.best_fit_allocator import BestFitAllocator
from allocators.first_fit_allocator import FirstFitAllocator
from allocators.worst_fit_allocator import WorstFitAllocator

In [2]:
# #test page
# bitmap = np.array([0,0,0,1,1,1,1,1,1, 0,1,0,1,0,1])
# is_zero = np.concatenate(([True], bitmap == 1, [True]))
# abs_diff = np.abs(np.diff(is_zero.astype(int)))

# ranges = np.where(abs_diff == 1)[0].reshape(-1, 2)
# fl = [{"idx": start, "size": end-start} for start, end in ranges]
# print(fl)

In [3]:
#test first fit
page = Page(page_size=16)
page.bitmap = np.array([0,0,0,1,1,1,1,1,1, 0,1,0,0,0,1,0])
print("initial bitmap: ", page.bitmap)
print("----------------------------------")

requests = [("a", 1), ("a", 2), ("a", 3), ("f", 1), ("f", 0), ("f", 11)]
first_fit_allocator = FirstFitAllocator()
first_fit_allocator.pages_in_use.append(page)
for req in requests:
    print("Request size: ", req)
    if req[0] == "a":
        page_idx, block_idx = first_fit_allocator.handle_alloc_req([page], req[1])
        print(f"Allocating at page {page_idx}, block {block_idx}")

        print("Allocation result: ", page.allocate(block_idx, req[1]))
    else:
        print("Freeing block: ", req[1])
        page.free(req[1])

print("----------------------------------")
print(page.bitmap)
print(page.free_list)
print(page.allocated_list)

initial bitmap:  [0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0]
----------------------------------
Request size:  ('a', 1)
In handle_alloc_req(), Handling allocation request of size:  1
Allocating at page 0, block 0
Allocation result:  True
Request size:  ('a', 2)
In handle_alloc_req(), Handling allocation request of size:  2
Allocating at page 0, block 1
Allocation result:  True
Request size:  ('a', 3)
In handle_alloc_req(), Handling allocation request of size:  3
Allocating at page 0, block 11
Allocation result:  True
Request size:  ('f', 1)
Freeing block:  1
Request size:  ('f', 0)
Freeing block:  0
Request size:  ('f', 11)
Freeing block:  11
----------------------------------
[0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0]
[{'idx': 0, 'size': 3}, {'idx': 9, 'size': 1}, {'idx': 11, 'size': 3}, {'idx': 15, 'size': 1}]
{}


In [4]:
#test best fit
page = Page(page_size=16)
page.bitmap = np.array([0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1])
page.free_list = [{"idx": 0, "size": 4}, {"idx": 5, "size": 2}, {"idx": 9, "size": 1}, {"idx": 6, "size": 3}]
print("initial bitmap: ", page.bitmap)
print("----------------------------------")

requests = [("a", 1), ("a", 3)]
best_fit_allocator = BestFitAllocator()
best_fit_allocator.pages_in_use.append(page)
for req in requests:
    print("Request size: ", req)
    if req[0] == "a":
        page_idx, block_idx = best_fit_allocator.handle_alloc_req([page], req[1])
        print(f"Allocating at page {page_idx}, block {block_idx}")

        print("Allocation result: ", page.allocate(block_idx, req[1]))
    else:
        print("Freeing block: ", req[1])
        page.free(req[1])

print("----------------------------------")
print(page.bitmap)
print(page.free_list)
print(page.allocated_list)

initial bitmap:  [0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1]
----------------------------------
Request size:  ('a', 1)
Allocating at page 0, block 9
Allocation result:  True
Request size:  ('a', 3)
Allocating at page 0, block 11
Allocation result:  True
----------------------------------
[0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1]
[{'idx': 0, 'size': 4}, {'idx': 5, 'size': 2}]
{9: 1, 11: 3}


In [5]:
#test worst fit
page = Page(page_size=16)
page.bitmap = np.array([0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1])
page.free_list = [{"idx": 0, "size": 4}, {"idx": 5, "size": 2}, {"idx": 9, "size": 1}, {"idx": 6, "size": 3}]
print("initial bitmap: ", page.bitmap)
print("----------------------------------")

requests = [("a", 1), ("a", 3)]
worst_fit_allocator = WorstFitAllocator()
worst_fit_allocator.pages_in_use.append(page)
for req in requests:
    print("Request size: ", req)
    if req[0] == "a":
        page_idx, block_idx = worst_fit_allocator.handle_alloc_req([page], req[1])
        print(f"Allocating at page {page_idx}, block {block_idx}")

        print("Allocation result: ", page.allocate(block_idx, req[1]))
    else:
        print("Freeing block: ", req[1])
        page.free(req[1])

print("----------------------------------")
print(page.bitmap)
print(page.free_list)

initial bitmap:  [0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1]
----------------------------------
Request size:  ('a', 1)
Allocating at page 0, block 0
Allocation result:  True
Request size:  ('a', 3)
Allocating at page 0, block 1
Allocation result:  True
----------------------------------
[1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1]
[{'idx': 5, 'size': 2}, {'idx': 9, 'size': 1}, {'idx': 11, 'size': 3}]


In [6]:
from environments.single_state_env import SingleStateEnv
from allocators.first_fit_allocator import FirstFitAllocator

env = SingleStateEnv()
s, done = env.reset()
print(s.keys(),s["bitmap"].shape, s["rq"], s['pages'],done)



<class 'request_streams.base_request_stream_dist.BaseRequestStreamDist'>
self.request_stream:  AllocationRequestStream: BaseRequestStreamDist
In get_next_req()
    free_or_alloc:  1 allocated_indices:  []
in _get_state()
    Request contents - free_or_alloc: 1, mem_addr_or_amt: 50, int(new_traj): 0)
dict_keys(['bitmap', 'rq', 'pages']) (1, 256) [ 1 50  0] [<page.Page object at 0x000001DE3231E970>] False


In [7]:
allocator = FirstFitAllocator()
r = -1
while not done:
    #ac is mem_addr_or_amt
    curr_page = s["pages"][0]
    print(print("Current page's free list: ", page.free_list, "Current page's allocated list: ", page.allocated_list))
    if s["rq"][0] == 0:
        ac = s["rq"][1]
    else:
        ac = allocator.handle_alloc_req(s["pages"], s["rq"][1])[1]

    s,r,done = env.step([s["rq"][0], ac])
    print("----------------------------------")

        

    Current page's free list:  [{'idx': 5, 'size': 2}, {'idx': 9, 'size': 1}, {'idx': 11, 'size': 3}] Current page's allocated list:  {0: 1, 1: 3}
None
In handle_alloc_req(), Handling allocation request of size:  50
In get_next_req()
    free_or_alloc:  0 allocated_indices:  [0]
in _get_state()
    Request contents - free_or_alloc: 0, mem_addr_or_amt: 0, int(new_traj): 0)
----------------------------------
    Current page's free list:  [{'idx': 5, 'size': 2}, {'idx': 9, 'size': 1}, {'idx': 11, 'size': 3}] Current page's allocated list:  {0: 1, 1: 3}
None
In get_next_req()
    free_or_alloc:  0 allocated_indices:  []
in _get_state()
    Request contents - free_or_alloc: 1, mem_addr_or_amt: 4, int(new_traj): 0)
----------------------------------
    Current page's free list:  [{'idx': 5, 'size': 2}, {'idx': 9, 'size': 1}, {'idx': 11, 'size': 3}] Current page's allocated list:  {0: 1, 1: 3}
None
In handle_alloc_req(), Handling allocation request of size:  4
In get_next_req()
    free_or_

AssertionError: Address 4 was never allocated!